In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 16487347
paper_name = 'ando_shima_2006' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_excel('raw_data/High-sugar stress.xlsx', sheet_name='high sucrose', skiprows=1)

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4715 x 5


In [10]:
original_data.head()

,ORF,GENE,Growth rate,Unnamed: 3,DESCRIPTION (MIPS)
0,NaN,NaN,YM/YW,SM/SW,NaN
1,YAL002W,VPS8,0.993138,0.908862,"vacuolar sorting protein, 134 kD"
2,YAL004W,NaN,1.05133,1.00879,questionable protein
3,YAL005C,SSA1,1.05262,1.10649,"heat shock protein of HSP70 family, cytosolic"
4,YAL007C,ERP2,1.12498,1.00918,p24 protein involved in membrane trafficking


In [11]:
original_data['orf'] = original_data['ORF'].astype(str)

In [12]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

                                                           ORF GENE  \
index_input                                                           
0                                                          NaN  NaN   
4714         ND, Not determined. See the reference for detail.  NaN   

            Growth rate Unnamed: 3 DESCRIPTION (MIPS)  \
index_input                                             
0                 YM/YW      SM/SW                NaN   
4714                NaN        NaN                NaN   

                                                 orf  
index_input                                           
0                                                NAN  
4714         NDNOTDETERMINEDSEETHEREFERENCEFORDETAIL  


In [15]:
original_data = original_data.loc[t,:]

In [16]:
original_data['data'] = pd.to_numeric(original_data['Unnamed: 3'], errors='coerce') / pd.to_numeric(original_data['Growth rate'], errors='coerce')

In [17]:
original_data.set_index('orf', inplace=True)

In [18]:
original_data = original_data[['data']].copy()

In [19]:
original_data = original_data.groupby(original_data.index).mean()

In [20]:
original_data.shape

(4688, 1)

# Prepare the final dataset

In [21]:
data = original_data.copy()

In [22]:
dataset_ids = [500]
datasets = datasets.reindex(index=dataset_ids)

In [23]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [24]:
data.head()

dataset_id,500
data_type,value
orf,
YAL002W,0.915142
YAL004W,0.959536
YAL005C,1.051176
YAL007C,0.897062
YAL008W,0.884174


## Subset to the genes currently in SGD

In [25]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [26]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,500
,data_type,value
gene_id,orf,
2,YAL002W,0.915142
1863,YAL004W,0.959536
4,YAL005C,1.051176
5,YAL007C,0.897062
6,YAL008W,0.884174


# Normalize

In [27]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [28]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [29]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id            500          
data_type           value    valuez
gene_id orf                        
2       YAL002W  0.915142 -0.484504
1863    YAL004W  0.959536 -0.282427
4       YAL005C  1.051176  0.134700
5       YAL007C  0.897062 -0.566797
6       YAL008W  0.884174 -0.625461

# Print out

In [30]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [31]:
from IO.save_data_to_db3 import *

In [32]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 16487347...
Inserting the new data...


100%|██████████| 1/1 [00:07<00:00,  7.07s/it]

Updating the data_modified_on field...
